In [1]:
import ipywidgets.widgets as widgets
from IPython.display import display
import cv2
import motors

# Create widgets for displaying images
display_color = widgets.Image(format='jpeg', width='45%')
display_depth = widgets.Image(format='jpeg', width='45%')
layout = widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth], layout=layout)
display(sidebyside)

# Convert numpy array to jpeg coded data for displaying
def bgr8_to_jpeg(value):
    return bytes(cv2.imencode('.jpg', value)[1])

In [2]:
from ultralytics import YOLO

# full model
#model = YOLO("yolo11l_half.engine")

model = YOLO("yolo11l_half.engine")


# human only model
# model = YOLO('best.pt')


WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.


In [3]:
import numpy as np

def detect_human_from_results(results, depth_frame, width, height):
    min_distance = float('inf')
    best_bbox = None
    best_center = None

    for result in results:
        for i, bbox in enumerate(result.boxes.xyxy):
            cx = int((bbox[0] + bbox[2]) / 2)
            cy = int((bbox[1] + bbox[3]) / 2)

            # Depth lookup bounds check
            if 0 <= cx < width and 0 <= cy < height:
                distance = depth_frame[cy, cx]

                if not np.isnan(distance) and distance > 0:
                    if distance < min_distance:
                        min_distance = distance
                        best_bbox = bbox
                        best_center = [cx, cy]

    return best_bbox is not None, min_distance, best_bbox, best_center

In [4]:
def pixel_to_angle(px, py, width, height, hfov, vfov):

    cx, cy = width / 2, height / 2

    h_angle = ((px - cx) / cx) * (hfov / 2)
    v_angle = ((py - cy) / cy) * (vfov / 2)

    return h_angle, v_angle

In [5]:
from camera import Camera 
from springfollower import SpringFollower

#create a camera object
cam = Camera()
cam.start() # start capturing the data

#create follower
follower = SpringFollower(speed=0.0, follow_distance=5.0, stopping_distance=2.0, v_max = 1.0, deadband=0.2)

[2025-11-21 10:08:59 UTC][ZED][INFO] Logging level INFO
[2025-11-21 10:08:59 UTC][ZED][INFO] Logging level INFO
Camera error: CAMERA NOT DETECTED


SystemExit: 1

[2025-11-21 10:08:59 UTC][ZED][INFO] Logging level INFO
[2025-11-21 10:09:00 UTC][ZED][WARNING] CAMERA NOT DETECTED in sl::ERROR_CODE sl::Camera::open(sl::InitParameters)


/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py:3587: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [ ]:
import time
import cv2

display_camera = True

run_time = range(300)
start_time = time.time()
yolo_interval = 5


latest_color_frame = None
latest_depth_frame = None

latest_results = None

# =========== Main Loop ===========
for i in run_time:
    # get current frame
    latest_color_frame, latest_depth_frame = cam.get_frame()

    # detect humans
    if run_time % yolo_interval == 0:
        latest_results = model(latest_color_frame, classes=[0], verbose=False)
    found, dist, bbox, center = detect_human_from_results(
        latest_results, latest_depth_frame, cam.width, cam.height
        )
    
    # calculate angle deviation from center
    angle = pixel_to_angle(
        center[0], center[1], cam.width, cam.height, cam.hfov, cam.vfov
    )

    if found:
        follower.update(angle, dist)



    # =========== Display images ===========
    
    if display_camera:
        # Prepare depth colormap for display
        depth_colormap = cv2.applyColorMap(
            cv2.convertScaleAbs(latest_depth_frame, alpha=0.03), 
            cv2.COLORMAP_JET)
        scale = 0.1
        resized_color = cv2.resize(latest_color_frame, None, fx=scale, fy=scale, 
                                   interpolation=cv2.INTER_AREA)
        resized_depth = cv2.resize(depth_colormap, None, fx=scale, fy=scale, 
                                   interpolation=cv2.INTER_AREA)
        display_color.value = bgr8_to_jpeg(resized_color)
        display_depth.value = bgr8_to_jpeg(resized_depth)



end_time = time.time()
elapsed_time = end_time - start_time

print("Full model inference on 300 frames completed.")
print(f"Elapsed Time: {elapsed_time} seconds")

control_frequency = len(run_time)/elapsed_time
print(f"Control frequency = ", control_frequency)



In [ ]:
#follower.stop()
#camera.stop()